In [ ]:
import scipy_differentialevolution_popsize_15d
import numpy as np
from algorithm import Algorithm
import numpy as np
import copy
from scipy.optimize import Bounds

class DE(Algorithm):
    """DE algorithm
    """
    __doc__ += Algorithm.__doc__
    
    def __init__(self, func):
        Algorithm.__init__(self, func)
        bounds = Bounds(self.func.lowerbound, self.func.upperbound)
        self.de_wrapper = scipy_differentialevolution_popsize_15d.DifferentialEvolutionSolver(self.func, bounds = bounds)

    def set_params(self, parameters):
        self.budget = parameters.budget
        
    def get_params(self, parameters):
        # do something
        return parameters
    
    def run(self):
        print(f'DE started')

        self.de_wrapper.maxfun = self.budget
        self.de_wrapper.stop = self.stop
        self.de_wrapper.solve()
        

        print(f'DE complete')
        print(f'evals: {self.func.evaluations} x_opt: {self.func.best_so_far_variables}')

        return self.func.best_so_far_variables, self.func.best_so_far_precision

In [ ]:
from IOHexperimenter import IOH_function, IOH_logger
import datetime
from shutil import make_archive
from IOH_function_wrapper import IOH_function_logger

# Experiment routine
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(
    now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
dimensions = [2, 3, 5, 10, 20]
instances = [1, 2, 3, 4, 5]
func_ids = []
for i in range(1,25):
    func_ids.append(i)
exp_budget = 10000
runs = 5
switch_combination = "DE_"
settings = "popsize_15D"
experiment_name = path + switch_combination + "/experiment" + experiment_ID + "_" + settings

for func_id in func_ids:
    for dim in dimensions:
        for inst in instances:
            f = IOH_function_logger(func_id, dim, inst, suite='BBOB')
            log = IOH_logger(experiment_name, "run",
                           switch_combination + settings, "switch-routine")
            f.add_logger(log)

            for run in range(0, runs):
                if run > 0:
                    f.reset()
                    f.x_hist = []
                    f.f_hist = []
                alg = DE(f)
                alg.budget = 10000 * f.number_of_variables
                tau = 1e-8
                func = alg.func
                budget = alg.budget
                def stopping_criteria():
                    return (func.best_so_far_precision) <= tau or (
                                    alg.func.evaluations >= budget)

                alg.set_stopping_criteria(stopping_criteria)
                x_opt, prec = alg.run()
                print(f' run {run} dim{dim} f{func_id} inst{inst} complete, precision: {prec}')
            f.clear_logger()

make_archive(experiment_name, "zip", experiment_name)